In [1]:
# Core
import numpy as np
import pandas as pd

# Astronomy
import rebound

In [2]:
# Local imports
import kepler_sieve
from db_utils import sp2df, df2db, sp_run
from astro_utils import mjd_to_datetime
from rebound_sim import make_sim_planets
from rebound_integrate import integrate_mjds, integrate_mjds_df
from ra_dec import calc_topos
from sky_patch import dir2SkyPatchID, SkyPatchID2dir
from ztf_data import calc_ztf_detection_times 
from enrich_detections import calc_detections

In [3]:
# Date range
mjd0: int = 58270
mjd1: int = mjd0 + 10

# SkyPatch grid size
N_sky_patch: int = 1024

In [6]:
# Assemble a batch of detections
df = calc_detections(mjd0=mjd0, mjd1=mjd1, N_sky_patch=N_sky_patch)

In [5]:
df

,DetectionTimeID,DetectionID,mjd,ra,dec,ux,uy,uz,mag,SkyPatchID,k
0,1,1,58270.167662,197.497479,9.661561,-0.940203,-0.205183,0.271880,17.028400,14222632,1
1,1,2,58270.167662,197.528147,9.462841,-0.940593,-0.207164,0.269011,20.005800,14218532,1
2,1,3,58270.167662,197.125117,9.456218,-0.942678,-0.201138,0.266274,19.185200,14232865,1
3,1,4,58270.167662,196.874981,9.368209,-0.944177,-0.198029,0.263275,19.778099,14241053,1
4,1,5,58270.167662,196.790613,9.262449,-0.944884,-0.197557,0.261085,19.809299,14241050,1
...,...,...,...,...,...,...,...,...,...,...,...
675138,910,676048,58279.457650,328.949154,49.864882,0.552229,-0.000933,0.833692,18.890301,3486718,1
675139,910,676049,58279.457650,328.822659,49.862018,0.551526,-0.002082,0.834155,18.224300,3484669,1
675140,910,676050,58279.457650,328.208625,49.975793,0.546627,-0.006244,0.837353,18.119600,3466232,1
675141,910,676051,58279.457650,329.031648,49.880229,0.552531,-0.000038,0.833492,18.830299,3486719,1


## Calculate Direction of the Detections from RA/DEC

In [3]:
# Date range
mjd0: int = 58270
mjd1: int = mjd0 + 10

In [4]:
# Get DataFrame of raw detections
params = {'mjd0':mjd0, 'mjd1': mjd1}
df = sp2df('KS.GetRawDetections', params)

In [5]:
# Calculate direction
ztf_detection_add_dir(df)

## Calculate SkyPatchID

In [6]:
# Nx3 array of directions
dir = df[['ux', 'uy', 'uz']].values
N: int = 1024

In [7]:
# Calculate the SkyPatchID
SkyPatchID = dir2SkyPatchID(dir, N)

In [8]:
# Add the SkyPatchID
df['SkyPatchID'] = SkyPatchID

In [9]:
# Add the field k for the detection number in each SkyPatch
df['k'] = df.groupby(['DetectionTimeID', 'SkyPatchID']).cumcount()+1

In [10]:
df

,DetectionTimeID,DetectionID,mjd,ra,dec,ux,uy,uz,mag,SkyPatchID,k
0,1,1,58270.167662,197.497479,9.661561,-0.940203,-0.205183,0.271880,17.028400,14222632,1
1,1,2,58270.167662,197.528147,9.462841,-0.940593,-0.207164,0.269011,20.005800,14218532,1
2,1,3,58270.167662,197.125117,9.456218,-0.942678,-0.201138,0.266274,19.185200,14232865,1
3,1,4,58270.167662,196.874981,9.368209,-0.944177,-0.198029,0.263275,19.778099,14241053,1
4,1,5,58270.167662,196.790613,9.262449,-0.944884,-0.197557,0.261085,19.809299,14241050,1
...,...,...,...,...,...,...,...,...,...,...,...
676048,910,676049,58279.457650,328.822659,49.862018,0.551526,-0.002082,0.834155,18.224300,3484669,1
676049,910,676050,58279.457650,328.208625,49.975793,0.546627,-0.006244,0.837353,18.119600,3466232,1
676050,910,676051,58279.457650,329.031648,49.880229,0.552531,-0.000038,0.833492,18.830299,3486719,1
676051,910,676052,58279.457650,328.586834,49.929302,0.549381,-0.003432,0.835565,18.100000,3476475,1


## Check Distance from SkyPatch Midpoint to Direction u

In [12]:
# Direction at midpoint of the sky patches
dir_mid = SkyPatchID2dir(SkyPatchID, N=N)

# Difference to observed direction
diff = np.linalg.norm(dir-dir_mid, axis=1)

# Mean and max difference
diff_mean = np.mean(diff)
diff_max = np.max(diff)

In [16]:
# Report results
print('Difference from quoted direction to center of SkyPatch: (deg)')
print(f'Mean: {diff_mean*360:6.4f}')
print(f'Max:  {diff_max*360:6.4f}')

Difference from quoted direction to center of SkyPatch: (deg)
Mean: 0.0978
Max:  0.2444


## Insert This batch into the SkyPatch

In [7]:
columns = ['DetectionTimeID', 'SkyPatchID', 'k', 'DetectionID', 'ux', 'uy', 'uz', 'mag']
df2db(df=df, schema='KS', table='Detection', columns=columns)

  0%|          | 0/1 [00:00<?, ?it/s]

## Get SkyPatch Data

In [6]:
# Get cube faces
cf = sp2df('KS.GetCubeFace')

In [7]:
cf

,CubeFaceID,CubeFaceCD,i,j1,j2,ci,alpha,beta,gamma
0,1,Z+,3,1,2,1.0,X,Y,Z
1,2,Y+,2,3,1,1.0,Z,X,Y
2,3,X+,1,2,3,1.0,Y,Z,X
3,4,X-,1,2,3,-1.0,Y,Z,X
4,5,Y-,2,3,1,-1.0,Z,X,Y
5,6,Z-,3,1,2,-1.0,X,Y,Z


In [30]:
# Get the SkyPatch DataFrame
sp = sp2df('KS.GetSkyPatch')

sp2df(sp_name) failed!
Traceback (most recent call last):
  File "/home/michael/Harvard/kepler-sieve/src/db_utils.py", line 74, in sp2df
    df = pd.read_sql(sql_stmt, conn)
  File "/home/michael/Anaconda/anaconda3-2020.11/envs/kepler/lib/python3.8/site-packages/pandas/io/sql.py", line 510, in read_sql
    return pandas_sql.read_query(
  File "/home/michael/Anaconda/anaconda3-2020.11/envs/kepler/lib/python3.8/site-packages/pandas/io/sql.py", line 1294, in read_query
    result = self.execute(*args)
  File "/home/michael/Anaconda/anaconda3-2020.11/envs/kepler/lib/python3.8/site-packages/pandas/io/sql.py", line 1162, in execute
    return self.connectable.execution_options().execute(*args, **kwargs)
  File "/home/michael/Anaconda/anaconda3-2020.11/envs/kepler/lib/python3.8/site-packages/sqlalchemy/engine/base.py", line 1123, in execute
    return meth(self, multiparams, params, _EMPTY_EXECUTION_OPTS)
  File "/home/michael/Anaconda/anaconda3-2020.11/envs/kepler/lib/python3.8/site-packages

In [10]:
sp

,SkyPatchID,f,i,j,x,y,z
0,0,0,0,0,-0.577256,-0.577256,0.577538
1,1,0,0,1,-0.577444,-0.576880,0.577726
2,2,0,0,2,-0.577632,-0.576503,0.577914
3,3,0,0,3,-0.577820,-0.576126,0.578102
4,4,0,0,4,-0.578008,-0.575749,0.578290
...,...,...,...,...,...,...,...
25165819,25165819,5,2047,2043,0.578008,0.575749,-0.578290
25165820,25165820,5,2047,2044,0.577820,0.576126,-0.578102
25165821,25165821,5,2047,2045,0.577632,0.576503,-0.577914
25165822,25165822,5,2047,2046,0.577444,0.576880,-0.577726


## Insert batch (Just Directions; SkyPatchID Calculated Later)

In [11]:
# Get date range to roll up
dts = sp2df('KS.GetRawDetectionDates')

In [17]:
mjd0 = dts.mjd0[0]
mjd1 = dts.mjd1[0]

In [19]:
mjd1

59296

In [40]:
columns = ['DetectionTimeID','DetectionID','ux', 'uy', 'uz', 'mag']
df2db(df=df, schema='KS', table='DetectionBatch', columns=columns)

  0%|          | 0/1 [00:00<?, ?it/s]

## Calculate SkyPatchID from the Directions

In [63]:
cf

,CubeFaceID,CubeFaceCD,i,j1,j2,ci,alpha,beta,gamma
0,1,Z+,3,1,2,1.0,X,Y,Z
1,2,Y+,2,3,1,1.0,Z,X,Y
2,3,X+,1,2,3,1.0,Y,Z,X
3,4,X-,1,2,3,-1.0,Y,Z,X
4,5,Y-,2,3,1,-1.0,Z,X,Y
5,6,Z-,3,1,2,-1.0,X,Y,Z


In [84]:
# Nx3 array of directions
dir = df[['ux', 'uy', 'uz']].values

In [86]:
SkyPatchID = dir2SkyPatchID(dir, N)

In [87]:
dir_sp = sp.loc[SkyPatchID, ['x', 'y', 'z']].values

In [88]:
dist = np.linalg.norm(diff, axis=1)

In [92]:
np.max(dist)*360.0

0.24436361252903707

In [93]:
np.mean(dist)*360.0

0.0977585093661368